In [20]:
'''%pip install --disable-pip-version-check \
        torch==1.13.1 \
        torchdata==0.5.1 --quiet

%pip install \
        transformers==4.27.2 \
        datasets==2.11.0 --quiet'''

'%pip install --disable-pip-version-check         torch==1.13.1         torchdata==0.5.1 --quiet\n\n%pip install         transformers==4.27.2         datasets==2.11.0 --quiet'

In [21]:
'''%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0  --quiet'''

'%pip install --upgrade pip\n%pip install --disable-pip-version-check     torch==1.13.1     torchdata==0.5.1 --quiet\n\n%pip install     transformers==4.27.2     datasets==2.11.0  --quiet'

In [3]:
from config import reddit_client_id, reddit_client_secret, reddit_user_agent

In [4]:
import praw
# Initialize the Reddit API wrapper
reddit = praw.Reddit(client_id=reddit_client_id,
                     client_secret=reddit_client_secret,
                     user_agent=reddit_user_agent)

# Replace 'python' with the name of the subreddit you want to pull posts from
subreddit = reddit.subreddit('CryptoCurrency')

for submission in reddit.subreddit("CryptoCurrency").hot(limit=10):
    print(submission.title)

#TODO: extract headings and top level comments for top n threads (hot n?)
#TODO: store this where? Patch as json to some store - blob?

AMA Announcement: Unveiling the Future of Decentralized AI and Software Development with Tau Net ($AGRS) - March 25th, 6pm UTC - 1500 USDT Giveaway
Tutorial: Bridging MOON to Arbitrum One via Celer (Celer Bridge is now live!)
Ethereum Could Top $14,000 Next Year Alongside Bitcoin Boom: Standard Chartered
I analyzed the Last 3 Bitcoin halvings here's what I think will happen after the 2024 halving
Cardano Gets First Fiat-Backed Stablecoin with USDM
Daily Crypto Discussion - March 25, 2024 (GMT+0)
This Week in Crypto Games: 'Crypto Valleys' Blasts Off, Immutable and Polygon Plot $100M Fund
Litecoin is breaking out of 11Y old LTC/BTC ratio downtrend, next 1W candle on 25th March 2024
Professional Shitposting
Skybridge Capital Founder Advises 'Act Like You're Dead With Your Bitcoin and Don't Sell' 


In [ ]:
'''%pip install emoji'''

In [213]:
# compile the dataset
# convert emojis to plain text before writing to csv
from emoji import demojize
import re


submissions_dataset = []
comments_dataset = []

for submission in reddit.subreddit("CryptoCurrency").top('day',limit=10):
    submissions_dataset.append([submission.id, submission.title, submission.created, submission.score])
    for top_level_comment in submission.comments:
            comments_dataset.append([top_level_comment.id, re.sub(r'\s+', ' ', demojize(top_level_comment.body)), top_level_comment.created_utc, top_level_comment.score, submission.id])

C:\Users\olgakh\AppData\Local\Temp\ipykernel_14216\1999565252.py:10: DeprecationWarning: Positional arguments for 'BaseListingMixin.top' will no longer be supported in PRAW 8.
Call this function with 'time_filter' as a keyword argument.
  for submission in reddit.subreddit("CryptoCurrency").top('day',limit=10):


In [214]:
import csv

submissions_filename = "reddit_submissions_data.csv"
comments_filename = "reddit_comments_data.csv"

'''
# Define the fields and rows
# Do this the first time writing to the files

submission_fields = ['ID', 'Title', 'Unixtime', 'Upvotes']
comment_fields = ['ID','Comment', 'Unixtime', 'Upvotes', 'SubmissionID']
'''


with open(submissions_filename, 'a') as csvfile:
    # Create a writer object
    csvwriter = csv.writer(csvfile)
    # Write the fields and rows to the file
    #csvwriter.writerow(submission_fields)
    csvwriter.writerows(submissions_dataset)
    
with open(comments_filename, 'a', encoding="utf-8") as csvfile:
    # Create a writer object
    csvwriter = csv.writer(csvfile)
    # Write the fields and rows to the file
    #csvwriter.writerow(comment_fields)
    csvwriter.writerows(comments_dataset)


In [219]:
import pandas as pd
submissions_df = pd.read_csv(submissions_filename)

In [220]:
comments_df = pd.read_csv(comments_filename)

Note: # From qualitative analysis: a comment doesn't often have multiple coins+strong sentiment associated with each one, usually it's one coin mention and discussion, it may mention other coins but the comment itself will center around one of them and only have strong sentiment associated with that one coin. Comments like "BTC is amazing, Etherium sucks, Doge is the best" are rare. 

In [315]:
# Cleaning:
# Converted emojis to word representations, removed newlines and tabs
# \s on reddit means sarcasm - how to best represent this? Important for sentiment analysis

# remove removed comments
comments_df=comments_df[comments_df["Comment"]!="[removed]"]
# remove admin bot disclaimer comments 
comments_df = comments_df[~comments_df["Comment"].str.contains("\[pros\]\(/r/CryptoCurrency/")]

#comments_df["Comment"].str.split(expand=True).stack().value_counts()
#comments_df["Comment"].replace('\s', '*SARCASM*')
comments_df = comments_df.dropna(axis=0, how='any')

In [316]:
comments_df

,ID,Comment,Unixtime,Upvotes,SubmissionID
1,kwdqtko,Nobody knows shit,1.711309e+09,331,1bmshtm
2,kwdpxif,Ethereum could also top 20k. Or go back to 700...,1.711308e+09,549,1bmshtm
3,kwdsan3,Wtf is the point of following this sub every p...,1.711309e+09,137,1bmshtm
4,kwdqa7o,Could.,1.711309e+09,15,1bmshtm
5,kwdv5ch,It will either go up or down says my crystal b...,1.711310e+09,37,1bmshtm
...,...,...,...,...,...
411,kwfv14l,Joining the crypto gambleverse to dreaming to ...,1.711339e+09,1,1bmi3xr
412,kwg5z9w,"yes it is spread, welcome to the world of hidd...",1.711345e+09,1,1bmi3xr
413,kwbqla5,Ran away from Binance the moment they stopped ...,1.711280e+09,0,1bmi3xr
414,kwboc7t,I had a balance of 980€ in BTC after buying 10...,1.711278e+09,-6,1bmi3xr


In [ ]:
# TODO: BTC/Bitcoin mentions - need to map name/abbreviation to the same coin
# Can potentially be done by the LLM
# If not, can scrape the coinwire page and write a function to map

In [49]:
#TODO: load the base model using tensorflow

# Loading the base model, the tokenizer and the config
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [419]:

model_name='google/flan-t5-base'
xl_model_name = 'google/flan-t5-xl'
#model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
xl_model = AutoModelForSeq2SeqLM.from_pretrained(xl_model_name)


config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

C:\Users\olgakh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\olgakh\.cache\huggingface\hub\models--google--flan-t5-xl. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin.index.json:   0%|          | 0.00/50.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/repos/16/41/16418edd56a7c42307a0f361531c01ee227a92a98628972bd433062c276dad7c/c43a96461e48c0218de822525f8ecd12a8318ed01364cb1ca595e9a238fa49d2?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model-00001-of-00002.bin%3B+filename%3D%22pytorch_model-00001-of-00002.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1711685592&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMTY4NTU5Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8xNi80MS8xNjQxOGVkZDU2YTdjNDIzMDdhMGYzNjE1MzFjMDFlZTIyN2E5MmE5ODYyODk3MmJkNDMzMDYyYzI3NmRhZDdjL2M0M2E5NjQ2MWU0OGMwMjE4ZGU4MjI1MjVmOGVjZDEyYTgzMThlZDAxMzY0Y2IxY2E1OTVlOWEyMzhmYTQ5ZDI%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=FoOeli8WQGb%7E6LC-FMaki0-uy69yH8PzGjeY-ArYKvlXXss2FjGHqop--saEkcw3FNkKJq6339DkczdG4887S7IR4Y8FsAbONf2njYwEjZxjHCcEof9RZuYFnsyZNnpn5

pytorch_model-00001-of-00002.bin:  11%|#         | 1.02G/9.45G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [51]:
# Tokenizer 
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, batched = True)

In [ ]:
#TODO: write a prompt

prompt = f'''For a given Reddit comment, extract the Cryptocurrency the comment mentions and the sentiment associated with that cryptocurrency. 
If there are multiple Cryptocurrencies mentioned in the comment, extract the main Cryptocurrency that the commenter is focusing on.
Classify the sentiment associated with this Cryptocurrency into positive, negative or neutral.

Comment: {}'''

In [420]:
comment_list = comments_df["Comment"][:10]
example = comment_list[2]
example

'Ethereum could also top 20k. Or go back to 700$. Or 7k. Or anything else'

In [411]:
processed_comments_zero_shot = []
for example in comment_list:
    prompt = f"""
    Give the Cryptocurrency that this Reddit comment mentions and the sentiment associated with this Cryptocurrency. Output should be in the format [Cryptocurrency, sentiment], e.g. [Bitcoin, negative].
    If the comment mentions multiple Cryptocurrencies, only extract the one that is the main focus of the comment.

    Example: 
    Comment: "I hate Bitcoin, the age of Bitcoin is over"
    Output: [Bitcoin, negative]
    
    Comment:
    {example}
    Output:
    """
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs['input_ids'], 
            )[0], 
            skip_special_tokens=True
    )
    processed_comments_zero_shot.append([example, output])
    
        

C:\Users\olgakh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [417]:
prompt = """What is Etherium?"""
inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs['input_ids'], 
        )[0], 
        skip_special_tokens=True
)
print(output)

etherium


In [412]:
pd.DataFrame(processed_comments_zero_shot)

,0,1
0,Nobody knows shit,"[Bitcoin, negative]"
1,Ethereum could also top 20k. Or go back to 700...,"[Bitcoin, negative]"
2,Wtf is the point of following this sub every p...,"[Bitcoin, negative]"
3,Could.,"[Bitcoin, negative]"
4,It will either go up or down says my crystal b...,"[Cryptocurrency, positive]"
5,tldr; Standard Chartered predicts Ethereum cou...,"[Bitcoin, positive]"
6,Source: trust me bro,"[Bitcoin, positive]"
7,"I think $37k ETH, $272k BTC","[Cryptocurrency, positive]"
8,Coin stock is probably the shining star for cr...,"[Cryptocurrency, positive]"
9,Pretty much everyone I know says eth $10k targ...,"[Cryptocurrency, positive]"


In [402]:
list(comments_df[comments_df.Comment.str.contains("\\s")]["Comment"])


['Nobody knows shit',
 'Ethereum could also top 20k. Or go back to 700$. Or 7k. Or anything else',
 'Wtf is the point of following this sub every post is the exact same unfounded prediction.',
 'It will either go up or down says my crystal ball :crystal_ball: with 100% accuracy',
 "tldr; Standard Chartered predicts Ethereum could reach $8,000 by the end of this year and possibly $14,000 by 2025, contingent on the approval of spot Ethereum exchange-traded funds (ETFs) by the SEC. The bank's optimism is partly due to Ethereum's recent upgrade, which is expected to significantly reduce transaction fees, making the network more competitive. Additionally, Standard Chartered also forecasts Bitcoin could hit $150,000 per coin by year-end, following the approval and popularity of Bitcoin ETFs. *This summary is auto generated by a bot and not meant to replace reading the original article. As always, DYOR.",
 'Source: trust me bro',
 'I think $37k ETH, $272k BTC',
 'Coin stock is probably the sh